In [1]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import collections
from collections import Counter


In [2]:
def collect_metrics(root_dir):

    depth = 2

    dict_res_roberta = dict()
    dict_res_camem = dict()
    dict_res_xlm = dict()
    
    for subdir, dirs, files in os.walk(root_dir):
        if subdir[len(root_dir):].count(os.sep) < depth:
            for file_ in files:
                if file_ == "xai_metrics.json":
                    with open(subdir + "/" + file_) as f:
                        d = json.load(f)
                        reg_log_odd = d["reg_log_odd"]
                        reg_anti_log_odd = d["reg_anti_log_odd"]
                        reg_comp = d["reg_comp"]
                        reg_suff = d["reg_suff"]
                        
                    model_type = subdir.split(os.sep)[-1].split("_")[0]
                    if model_type == "roberta":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_roberta[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}
                    elif model_type == "camem":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_camem[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}
                    elif model_type == "xlm":
                        dataset_name = " ".join(subdir.split(os.sep)[-1].split("_")[1:])
                        dict_res_xlm[dataset_name] = {"reg log odd": reg_log_odd, "reg anti log odd": reg_anti_log_odd, "reg comp": reg_comp, "reg suff": reg_suff}

    return dict_res_roberta, dict_res_camem, dict_res_xlm

def _average_dicts(dicts):
    avg_dict = collections.defaultdict(lambda: 0)
    
    for i, d in enumerate(dicts):
        for k, v in d.items():
            avg_dict[k] += v
        
    for k, v in avg_dict.items():
        avg_dict[k] /= (i+1)
    
    return avg_dict

def average_over_users(res):
    averaged_en = collections.defaultdict(list)
    averaged_it = collections.defaultdict(list)

    for k, v in res.items():
        if "en" in k and not ("nf" in k):
            averaged_en[" ".join(k.split(" ")[0:2])].append(v)
        elif "it" in k and not ("nf" in k):
            averaged_it[" ".join(k.split(" ")[0:2])].append(v)
    
    result_en = dict()
    result_it = dict()
            
    for k, v in averaged_en.items():
        result_en["avg en " + k] = _average_dicts(v)
        
    for k, v in averaged_it.items():
        result_it["avg it " + k] = _average_dicts(v)
                        
    return result_en, result_it

def process_to_present(res, column_width="1.1cm", plot_transpose=True):
    df_res = pd.DataFrame.from_dict(res).T
    # df_res = df_res.rename(columns=mapping_columns_names)
    if not plot_transpose:
        df_res = df_res.T
    s = df_res.style
    s.format(na_rep='MISS', precision=2)
    print(s.to_latex(column_format='l'+('p{'+f'{column_width}'+'}')*len(df_res.columns)))

In [3]:
dict_res_roberta, dict_res_camem, dict_res_xlm = collect_metrics("../output/complexity_10_50")

## ROBERTA

In [4]:
avg, _ = average_over_users(dict_res_roberta)
roberta_results = {**dict_res_roberta, ** avg}

In [5]:
pd.DataFrame(roberta_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
np f en6,0.164089,0.374693,0.0,0.0
p f en98,0.101873,0.318649,0.0,0.0
p f en6,0.109399,0.321362,0.0,0.0
p f en83,0.097590,0.243928,0.0,0.0
np f en57,0.050132,1.000946,0.0,0.0
p f en49,0.085953,0.218740,0.0,0.0
p f en57,0.107597,0.279033,0.0,0.0
np f en83,0.140064,1.105205,0.0,0.0
np f en49,0.050524,0.969469,0.0,0.0
np f en98,0.053310,0.896614,0.0,0.0


In [6]:
process_to_present(roberta_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
np f en6 & 0.16 & 0.37 & 0.00 & 0.00 \\
p f en98 & 0.10 & 0.32 & 0.00 & 0.00 \\
p f en6 & 0.11 & 0.32 & 0.00 & 0.00 \\
p f en83 & 0.10 & 0.24 & 0.00 & 0.00 \\
np f en57 & 0.05 & 1.00 & 0.00 & 0.00 \\
p f en49 & 0.09 & 0.22 & 0.00 & 0.00 \\
p f en57 & 0.11 & 0.28 & 0.00 & 0.00 \\
np f en83 & 0.14 & 1.11 & 0.00 & 0.00 \\
np f en49 & 0.05 & 0.97 & 0.00 & 0.00 \\
np f en98 & 0.05 & 0.90 & 0.00 & 0.00 \\
np nf en & 0.05 & 1.22 & 0.00 & 0.00 \\
p nf en & 0.09 & 0.33 & 0.00 & 0.00 \\
avg en np f & 0.09 & 0.87 & 0.00 & 0.00 \\
avg en p f & 0.10 & 0.28 & 0.00 & 0.00 \\
\end{tabular}



## CAMEM

In [7]:
_, avg = average_over_users(dict_res_camem)
camem_results = {**dict_res_camem, ** avg}

In [8]:
pd.DataFrame(camem_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
np nf it,0.245296,0.729543,0.0,0.0
np f it1,0.287339,0.617103,0.0,0.0
np f it26,0.335181,0.956257,0.0,0.0
np f it44,0.441073,0.335317,0.0,0.0
np f it43,0.201940,1.012082,0.0,0.0
p f it38,0.086263,0.419557,0.0,0.0
p f it43,0.132620,0.336977,0.0,0.0
p f it44,0.117918,0.265823,0.0,0.0
p f it26,0.139998,0.274145,0.0,0.0
np f it38,0.160101,1.043719,0.0,0.0


In [9]:
process_to_present(camem_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
np nf it & 0.30 & 0.33 & 0.62 & 0.72 \\
np f it1 & 0.35 & 0.90 & 0.60 & 1.68 \\
np f it26 & 0.43 & 1.03 & 0.72 & 1.64 \\
np f it44 & 0.61 & 0.34 & 0.43 & 0.80 \\
np f it43 & 0.25 & 0.76 & 0.61 & 1.17 \\
p f it38 & 0.15 & 0.65 & 0.37 & 0.70 \\
p f it43 & 0.21 & 0.39 & 0.41 & 0.69 \\
p f it44 & 0.20 & 0.56 & 0.38 & 1.06 \\
p f it26 & 0.27 & 0.33 & 0.47 & 1.03 \\
np f it38 & 0.18 & 0.83 & 0.64 & 0.54 \\
p f it1 & 0.31 & 0.49 & 0.39 & 0.58 \\
p nf it & 0.21 & 0.36 & 0.35 & 0.69 \\
avg it np f & 0.36 & 0.77 & 0.60 & 1.17 \\
avg it p f & 0.23 & 0.48 & 0.40 & 0.81 \\
\end{tabular}



## XLM

In [10]:
avg_en, avg_it = average_over_users(dict_res_xlm)
xlm_results = {**dict_res_xlm, **avg_en, **avg_it}

In [11]:
pd.DataFrame(xlm_results).T

,reg log odd,reg anti log odd,reg comp,reg suff
p nf en,0.204628,0.424284,0.316101,1.094665
np f en49,0.263090,0.565619,0.609430,0.788585
np f it1,0.195089,0.721184,0.548888,1.124371
p f en83,0.228019,0.326381,0.276982,0.840931
np nf it,0.117719,0.796314,0.649633,0.619584
p f en49,0.306161,0.763289,0.287636,1.840086
np f en83,0.163308,0.811359,0.545562,0.842169
p f en98,0.182010,0.243542,0.305199,0.714898
np f en98,0.244888,0.343191,0.603611,1.387963
np f en6,0.388269,0.770802,0.866314,1.314582


In [12]:
process_to_present(xlm_results)

\begin{tabular}{lp{1.1cm}p{1.1cm}p{1.1cm}p{1.1cm}}
 & reg log odd & reg anti log odd & reg comp & reg suff \\
p nf en & 0.20 & 0.42 & 0.32 & 1.09 \\
np f en49 & 0.26 & 0.57 & 0.61 & 0.79 \\
np f it1 & 0.20 & 0.72 & 0.55 & 1.12 \\
p f en83 & 0.23 & 0.33 & 0.28 & 0.84 \\
np nf it & 0.12 & 0.80 & 0.65 & 0.62 \\
p f en49 & 0.31 & 0.76 & 0.29 & 1.84 \\
np f en83 & 0.16 & 0.81 & 0.55 & 0.84 \\
p f en98 & 0.18 & 0.24 & 0.31 & 0.71 \\
np f en98 & 0.24 & 0.34 & 0.60 & 1.39 \\
np f en6 & 0.39 & 0.77 & 0.87 & 1.31 \\
p f it38 & 0.20 & 0.30 & 0.24 & 1.28 \\
np f it38 & 0.26 & 0.35 & 0.52 & 0.95 \\
np f it26 & 0.27 & 0.74 & 0.47 & 1.70 \\
np f it44 & 0.25 & 0.99 & 0.52 & 1.69 \\
np f it43 & 0.34 & 2.00 & 0.67 & 1.87 \\
p f it1 & 0.30 & 0.40 & 0.36 & 2.04 \\
p f it26 & 0.30 & 0.30 & 0.28 & 1.06 \\
p f it44 & 0.16 & 0.22 & 0.23 & 0.94 \\
p f it43 & 0.13 & 0.17 & 0.18 & 0.81 \\
p nf it & 0.16 & 0.23 & 0.20 & 0.97 \\
np nf en & 0.18 & 0.79 & 0.66 & 0.75 \\
p f en57 & 0.22 & 0.34 & 0.23 & 0.95 \\
p f en